In [75]:
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

df = pd.read_parquet('./frame.parquet.gzip')
xdf = df[['title', 'tags', 'description', 'publish_hour', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'channel_title']]
ydf = df[['category_id']]
Xtr, Xts, ytr, yts = train_test_split(xdf, ydf, shuffle=True, test_size=0.33)

Num GPUs Available:  1


In [73]:
def convert_to_sequences(x):
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(x)
    _sequences = tokenizer.texts_to_sequences(x)
    maxlen = max(len(_seq) for _seq in _sequences)
    vocab_size = len(tokenizer.word_index) + 1
    _xtr = tf.keras.preprocessing.sequence.pad_sequences(_sequences, maxlen, padding='post')
    
    return _xtr, maxlen, vocab_size

def create_text_channel(maxlen, vocab_size):
    input_ = tf.keras.layers.Input(shape=(maxlen,))
    embedding = tf.keras.layers.Embedding(vocab_size, 1028)(input_)
    conv = tf.keras.layers.Conv1D(filters=32, kernel_size=4, activation='softmax')(embedding)
    drop = tf.keras.layers.Dropout(0.05)(conv)
    pool = tf.keras.layers.MaxPooling1D(pool_size=2)(drop)
    flat = tf.keras.layers.Flatten()(pool)
    
    return input_, flat

def create_numerical_channel(shape=(3,)):
    input_ = tf.keras.layers.Input(shape=shape)
    dense = tf.keras.layers.Dense(8, activation='softmax')(input_)
    dense = tf.keras.layers.Dense(6, activation='softmax')(input_)
    #conv = tf.keras.layers.Conv1D(filters=10, kernel_size=4, activation='softmax')(dense)
    drop = tf.keras.layers.Dropout(0.05)(dense)
    flat = tf.keras.layers.Flatten()(drop)
    
    return input_, drop


def create_model(maxlen_vocabsz, tchannels=3, nchannels=3):
    inputs = []
    merge = []
    
    for maxlen, vocab_size in maxlen_vocabsz:
        for _ in range(tchannels):
            input_, flat = create_text_channel(maxlen, vocab_size)
            inputs.append(input_)
            merge.append(flat)
            
    for _ in range(nchannels):
        input_, flat = create_numerical_channel(shape=(3,))
        inputs.append(input_)
        merge.append(flat)
    
    merged = tf.keras.layers.Concatenate()(merge)
    dense = tf.keras.layers.Dense(10, activation='relu')(merged)
    outputs = tf.keras.layers.Dense(1, activation='relu')(dense)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    
    return model

In [84]:
maxlen_vocabsz = []
Xtrain = []

tchannels = 1
for feat in ['title', 'tags', 'description', 'channel_title']:
    _xtr, _maxlen, _vocab_size = convert_to_sequences(Xtr[feat].to_list())
    maxlen_vocabsz.append((_maxlen, _vocab_size))
    for _ in range(tchannels):
        Xtrain.append(np.array(_xtr))

nfeats = ['publish_hour', 'comments_disabled', 'ratings_disabled']
nchannels = 3
for _ in range(nchannels):
    Xtrain.append(Xtr[nfeats].to_numpy())

In [85]:
model = create_model(maxlen_vocabsz, tchannels=tchannels, nchannels=nchannels)
model.fit(Xtrain, ytr.to_numpy(), epochs=3, batch_size=16)

Epoch 1/3
1691/1691 [==============================] - 53s 31ms/step - loss: -55.5922 - accuracy: 0.2444
Epoch 2/3
1691/1691 [==============================] - 53s 31ms/step - loss: -55.6464 - accuracy: 0.2445
Epoch 3/3
1691/1691 [==============================] - 53s 31ms/step - loss: -55.6465 - accuracy: 0.2445


In [86]:
model.save('model2.h5')